## CS:APP Chapter2 homework

Note:<img src="./chapter2/premise.png" width="70%">

<img src="./chapter2/2-55.png" align="left" width="80%">

In [6]:
#include <stdio.h>
typedef unsigned char *byte_pointer;
void show_bytes(byte_pointer start,int len) {
    int i;
    for(i=0;i<len;i++) {
        printf(" %.2x",start[i]);
    }
    printf("\n");
}
void show_int(int x){
    show_bytes((byte_pointer)&x,sizeof(int));
}
int main() {
    show_int(0x2345);
}
// 根据结果，说明该机器采用了“小端法”来表示多字节的整数

 45 23 00 00


<img src="./chapter2/2-56.png" align="left" width="45%">

In [8]:
#include <stdio.h>
typedef unsigned char *byte_pointer;
void show_bytes(byte_pointer start,int len) {
    int i;
    for(i=0;i<len;i++) {
        printf(" %.2x",start[i]);
    }
    printf("\n");
}
void show_int(int x){
    show_bytes((byte_pointer)&x,sizeof(int));
}
void show_float(float x) {
    show_bytes((byte_pointer)&x,sizeof(float));
}
int main() {
    show_int(0x2345);
    show_float(0x2345);
}

 45 23 00 00
 00 14 0d 46


<img src="./chapter2/2-57.png" align="left" width="80%">

In [10]:
#include <stdio.h>
typedef unsigned char *byte_pointer;
void show_bytes(byte_pointer start,int len) {
    int i;
    for(i=0;i<len;i++) {
        printf(" %.2x",start[i]);
    }
    printf("\n");
}
void show_short(short x) {
    show_bytes((byte_pointer)&x,sizeof(short));
}
void show_long(long x) {
    show_bytes((byte_pointer)&x,sizeof(long));
}
void show_double(double x) {
    show_bytes((byte_pointer)&x,sizeof(double));
}
int main() {
    show_short(0x2345);
    show_long(0x2345);
    show_double(0x2345);
}
// 结果说明在该机器上'short'为2字节；'long'为4字节；'double'为8字节

 45 23
 45 23 00 00
 00 00 00 00 80 a2 c1 40


<img src="./chapter2/2-58.png" align="left" width="80%">

In [10]:
#include <stdio.h>
typedef unsigned char *byte_pointer;
int is_little_endian() {//条件：字长至少为1字节
    int p=0x12345678;
    if((int)((byte_pointer)&p)[0]==(p&0xff)) {
        return 1;
    } else {
        return 0;
    }
}
int main() {
    printf("%d\n",is_little_endian());
}

1


In [17]:
#include <stdio.h>
int is_little_endian() {
    int a=1;
    return *((char *)&a);
}
int main() {
    printf("%d\n",is_little_endian());
}

1


<img src="./chapter2/2-59.png" align="left" width="80%">

In [18]:
#include <stdio.h>
int combine(int x,int y) {
    return (y& ~0xff)|(x&0xff);
}
int main() {
    printf("%x\n",combine(0x89ABCDEF,0x76543210));
}

765432ef


<img src="./chapter2/2-60.png" align="left" width="80%">

In [26]:
#include <stdio.h>
unsigned put_byte(unsigned x,unsigned char b,int i) {
    return (x & ~(0xff<<(i<<3))) | (b<<(i<<3)); // char类型自动扩展
}
int main() {
    printf("%x\n",put_byte(0x12345678,0xab,3));
}

ab345678


NOTE:<img src="./chapter2/rule.png" align="center" width="80%">

<img src="./chapter2/2-61.png" align="left" width="80%">

In [27]:
#include <stdio.h>

int A(int x) {
    return !~x;
}

int B(int x) {
    return !x;
}

int C(int x) {
    return ! ~(x>>((sizeof(x)-1)<<3)); //算术位移
}

int D(int x) {
    return !(x&0xff);
}

int main() {
    int x=0xffffffff;
    printf("%d\n",A(x));
    x=0x00;
    printf("%d\n",B(x));
    x=0xffff1100;
    printf("%d\n",C(x));
    x=0x001100;
    printf("%d\n",D(x));
}

1
1
1
1


<img src="./chapter2/2-62.png" align="left" width="80%">

In [31]:
#include <stdio.h>
int int_shifts_are_logical() {
    int x=0xff;
    int shift=(sizeof(int)-1)<<3;
    int y = x<<shift>>shift;
    return y!=x;
    
}
int main() {
    printf("%d\n",int_shifts_are_logical());
}

1


In [28]:
#include <stdio.h>
int int_shifts_are_logical() {
    int x=-1;
    return x>>1 == -1;
}
int main() {
    printf("%d\n",int_shifts_are_logical());
}


1


<img src="./chapter2/2-63.png" align="left" width="80%">

In [39]:
#include <stdio.h>
/*
对于sra，主要的工作是将xrsl的第w-k-1位扩展到前面的高位。 这个可以利用取反加1来实现，不过这里的加1是加1<<(w-k-1)。 
如果x的第w-k-1位为0，取反加1后，前面位全为0，如果为1，取反加1后就全是1。 最后再使用相应的掩码得到结果。
*/
int sra(int x,int k) {
    int xsrl=(unsigned)x>>k;
    int w=sizeof(int)<<3;
    int z=1<<(w-k-1);
    int mask=z-1;
    int right = mask & xsrl;
    int left = ~mask & ~(z&xsrl)+z;
    return left | right ;
}
int srl(unsigned x,int k) {
    unsigned xsra=(int)x>>k;
    int w=sizeof(int)<<3;
    return xsra & ~(-2<<(w-k-1));
}
int main() {
    int x=0x8f000000;
    printf("%08x\n",srl(x,0));//空位补0
    printf("%08x\n",sra(x,2));//空位补最高位值
}

8f000000
e3c00000


<img src="./chapter2/2-64.png" align="left" width="80%">

In [34]:
#include <stdio.h>
int any_even_one(unsigned x) {
    int flag=0xaaaaaaaa;
    return !(x & flag ^ flag);
}
int main() {
    printf("%d\n",any_even_one(0xffffffff));
}

1


<img src="./chapter2/2-65.png" align="left" width="80%">

In [ ]:
#include <stdio.h>
int even_one(unsigned x) {
    unsigned int val=0;
    unsigned int flag=0x01010101;
    val += x&flag;
    flag<<=1;
}

<img src="./chapter2/2-66.png" align="left" width="80%">

<img src="./chapter2/2-67.png" align="left" width="80%">

In [ ]:
/*
A. 左移或右移的偏移量在[0,w-1]区间中
*/


<img src="./chapter2/2-68.png" align="left" width="80%">

In [5]:
#include <stdio.h>
int lower_bits(int x,int n) {
    int w=sizeof(x)<<3;
    int flag=1<<(w-1);
    printf("%x\n",flag);
    flag>>=(-n);
    printf("%x\n",flag);
    return x & (~flag);
}
int main() {
    int x=0x78abcdef;
    int n=32;
    printf("%x\n",lower_bits(x,n));
}

80000000
80000000
78abcdef


<img src="./chapter2/2-69.png" align="left" width="80%">

<img src="./chapter2/2-70.png" align="left" width="80%">

In [42]:
#include <stdio.h>
int main() {
    int a[5],b;
    a[1]=3;
    b=2;
}